In [1]:
%pip install python-dotenv

import dotenv

import enclave.models


# change to the env of your choice that matches the API keys: PROD, DEV, SANDBOX etc.
BASE_URL = enclave.models.WS_SANDBOX
env_filename = "sandbox.env"

API_KEY: str = ""
API_SECRET: str = ""


if not all([API_KEY, API_SECRET]):
    envs = dotenv.dotenv_values(env_filename)
    if envs and "key" in envs and "secret" in envs:
        API_KEY, API_SECRET = str(envs["key"]), str(envs["secret"])

if not all([API_KEY, API_SECRET]):
    raise ValueError("Please provide API_KEY and API_SECRET")

Note: you may need to restart the kernel to use updated packages.


In [2]:
connect_callback = lambda : print("On Connect")
auth_callback = lambda : print("On Auth")
error_callback = lambda x: print(f"On Error: {x}")
disconnect_callback = lambda : print("On Disconnect")
exit_callback = lambda : print("On Exit")

In [3]:
from enclave import wsclient

ws = wsclient.WebSocketClient(API_KEY, API_SECRET, base_url=BASE_URL, on_connect=connect_callback, on_auth=auth_callback, on_error=error_callback, on_disconnect=disconnect_callback, on_exit=exit_callback)

In [4]:
ws.add_pending_subscription("deposits", print)

In [5]:
import asyncio
from typing import Any

async def sleep_do(sleep_secs: float, do: Any):
    await asyncio.sleep(sleep_secs)
    await do()

async def close_after(ws: wsclient.WebSocketClient, seconds: float):
    await asyncio.sleep(seconds)
    await ws.close()

async def subscribe_and_unsubscribe_prices(sleep_secs: float = 5):
    await asyncio.sleep(sleep_secs)
    await ws.subscribe_callback("prices", lambda x: print(f"On Prices: {x}"))
    await asyncio.sleep(sleep_secs)
    await ws.unsubscribe_callback("prices")

In [6]:
# def auth_callback(calls=0):
#     def cb():
#         nonlocal calls
#         if calls == 0:
#             calls += 1
#             raise ValueError("simulate auth error the first time")
#         print("On Auth")
#     return cb

In [7]:
await asyncio.gather(ws.run(), close_after(ws, 20), subscribe_and_unsubscribe_prices(1))

On Connect
On Auth
On Prices: {'type': 'update', 'channel': 'prices', 'data': {'pair': {'base': 'AAVE', 'quote': 'USDC'}, 'price': '98.8896624500350105', 'time': '2023-12-07T22:29:22.742943479Z'}}
On Prices: {'type': 'update', 'channel': 'prices', 'data': {'pair': {'base': 'WBTC', 'quote': 'USDCa'}, 'price': '43347.0085512352205662', 'time': '2023-12-07T22:29:22.742943479Z'}}
On Prices: {'type': 'update', 'channel': 'prices', 'data': {'pair': {'base': 'USDT', 'quote': 'USD'}, 'price': '1.0001625', 'time': '2023-12-07T22:29:22.742943479Z'}}
On Prices: {'type': 'update', 'channel': 'prices', 'data': {'pair': {'base': 'AVAX', 'quote': 'USDCa'}, 'price': '26.6154834473592078', 'time': '2023-12-07T22:29:22.742943479Z'}}
On Prices: {'type': 'update', 'channel': 'prices', 'data': {'pair': {'base': 'AVAX', 'quote': 'USDC'}, 'price': '26.6154834473592078', 'time': '2023-12-07T22:29:22.742943479Z'}}
On Prices: {'type': 'update', 'channel': 'prices', 'data': {'pair': {'base': 'AAVE', 'quote': 'WB

[True, None, None]